In [2]:
import pandas as pd
from stemming.porter2 import stem
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

import random
import numpy as np  
from sklearn import decomposition
from sklearn.manifold import TSNE

import bokeh.plotting as bp
from bokeh.plotting import save
from bokeh.models import HoverTool
import re

In [3]:
file = pd.read_csv('Complaint_Data.csv')

In [4]:
issue_date = []
Rep = []
Issues = []
for rows in file['C1. Issue Desc Dtl']:
    a = str(rows)
    start_position_date = a.find("dt: ")
    end_position_date = a.find("tm: ")
    issue_date.append(a[start_position_date + len("dt: ") : end_position_date - 1])
    
    start_position_Rep = a.find("ISSUE: Created by  ")
    end_position_Rep = a.find(" ( logon id:", start_position_Rep)
    Rep.append(a[start_position_Rep + len("ISSUE: Created by  "): end_position_Rep])
    
    start_position_issue = a.find(" .", start_position_Rep)
    end_position_issue = a.find("}", start_position_issue)
    Issues.append(a[start_position_issue + len(" .") : end_position_issue])

## Processing
- Replace some shortcut based on the list you give.  
- Convert to lower case for everyword
- If the length of the entry is less than 5, delete

In [30]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.lower().replace(i, j)
    return text

In [31]:
def key_word_digit_extract(string):
    key_words=[]
    pattern=re.compile('.*?([0-9]+).*?')
    results=re.findall(pattern,string)
    if(len(results)==0):
        return key_words
    words=string.split(' ')
    for word in words:
        result=re.findall(pattern,word)
        if(len(result)>0):
            key_words.append(word)
    return key_words

In [32]:
replaces_list = {'\\&quot;':'', '\\&apos;':"'", "cust ": "customer ", "\n", ""}

In [45]:
## Extract the digit_relevant content out of the content
for x in Issues:
    key_words += key_word_digit_extract(x)

In [46]:
Issue_1 = []
for i in range(0, len(Issues)):
    if len(Issues[i].split(" ")) >= 5:
        Issue_1.append(replace_all(Issues[i], replaces_list))
    else: 
        i += 1

In [54]:
for i in range(0, len(Issue_1)):
    for word in Issue_1[i].split(" "):
        print (word)
        if word in key_words:
            word.replace(word, "")
        else:
            word
        Issue_2 = " ".join(word)

customer
left
debit
card
in
atm
nicole
left
her
debit
card
in
her
office
desk
and
called
pnc
to
have
it
suspended
the
card
status
was
lost
and
it
would
not
let
me
remove
the
warm
code
due
to
the
account
being
inactive.
nicole
was
extremely
upset
as
she
said
she's
done
this
in
the
past.
after
hanging
up
with
nicole
i
did
speak
with
a
partner
who
showed
me
a
way
to
get
around
it,
reactivate
the
card,
then
restates
it
from
the
lost
status.
i
was
able
to
complete
this
but
when
i
reached
out
to
nicole
she
had
already
destroyed
the
card.
instructed
her
to
go
to
a
branch
and
have
a
new
one
printed
and
that
i
would
order
her
a
new
pirates
card.
customer
need
help
activating
card.
customer
dislikes
that
there
is
no
branches
in
her
area
that
give
instant
debit
card
availability.
claiming
that
she
cannot
be
without
her
debit
card.
inquired
debit
card
status
that
was
ordered
6-9-17.
customer
called
in
because
she
opened
her
account
on
9/7
and
has
not
gotten

her
debit
card-
advised
that
it
may
sti

In [53]:
Issue_2

't o d a y .'

## Auto Correnction

In [ ]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words("".join(Issue_1)))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
P('dbit')

## STEMMING

In [ ]:
# Comment out
# import Stemer.py
import sys
print(sys.path)

In [ ]:
Issue_2 = [" ".join([stem(word) for word in sentence.split(" ")]) for sentence in Issue_1]

# LDAs

In [ ]:
stopword = ['a', 'the', 'customer', 'client', 'debit', 'card', 'this', 'that', 'to', 'his', 'her', 'he', 'she', 'it', 'is', 'are'
'and', 'but', 'custom' ]

In [ ]:

def print_topic_words(datainput, method = "NMF", num_topics = 20, num_top_words = 15, max_iteration = 10):
    topic_words = []
    tf = TfidfVectorizer(analyzer='word', 
                     ngram_range=(1, 1), 
                     max_df = 0.9, 
                     min_df= 5 , 
                     norm = 'l1',
                     stop_words='english')
    tfidf = tf.fit_transform(datainput)
    vocab = np.array(tf.get_feature_names())
    
    if method == "NMF":
        clf = decomposition.NMF(n_components=num_topics, random_state=1, alpha=.1, l1_ratio=.5)
        doctopic = clf.fit_transform(tfidf)
        
    elif method == "LDA":
        clf = decomposition.LatentDirichletAllocation(n_topics=num_topics, max_iter= max_iteration,
                                learning_method = 'batch'
#                                ,learning_offset = 50.
#                                ,random_state=1
                                                     )
        doctopic = clf.fit_transform(tfidf)
    
    for topic in clf.components_:
        word_idx = np.argsort(topic)[::-1][0:num_top_words]
        topic_words.append([vocab[i] for i in word_idx])
    
    return topic_words, doctopic

In [ ]:
## user input
num_topics = 20
num_top_words = 30
max_iteration = 100

In [ ]:
topic_words, doctopic = print_topic_words(Issue_2, method = "LDA", num_topics = num_topics, num_top_words = num_top_words)

In [ ]:
reordered = []
doctopic_weights = np.sum(doctopic, axis=0)
ranking = np.argsort(doctopic_weights)[::-1]  

for i, x in enumerate(ranking):
    reordered.append(topic_words[x])   
    print("Topic {}: {}".format(ranking[i], ' '.join(reordered[i][: num_top_words])))

In [ ]:
## For each complaints in the lsit, what is the most likely topic it will fall under? 
import numpy

threshold = 0.5
_idx = np.amax(doctopic, axis=1) > threshold  # idx of doc that above the threshold
_topics = doctopic[_idx]
# a t-SNE model
# angle value close to 1 means sacrificing accuracy for speed
# pca initializtion usually leads to better results 
tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')

# 20-D -> 2-D
tsne_lda = tsne_model.fit_transform(_topics)

In [ ]:
_lda_keys = []
for i in range(_topics.shape[0]):
    _lda_keys.append( _topics[i].argmax())

In [ ]:
# use the coordinate of a random tweet as string topic string coordinate
topic_coord = np.empty((doctopic.shape[1], 2)) * np.nan
for topic_num in _lda_keys:
    if not np.isnan(topic_coord).any():
        break
    topic_coord[topic_num] = tsne_lda[_lda_keys.index(topic_num)]

In [ ]:
colormap = np.array([
    "#1f77b4", "#aec7e8", "#ff7f0e", "#ffbb78", "#2ca02c",
    "#98df8a", "#d62728", "#ff9896", "#9467bd", "#c5b0d5",
    "#8c564b", "#c49c94", "#e377c2", "#f7b6d2", "#7f7f7f",
    "#c7c7c7", "#bcbd22", "#dbdb8d", "#17becf", "#9edae5"
])


title = '20 Topics LDA viz'

num_example = len(doctopic)

plot_lda = bp.figure(plot_width=1400, plot_height=1100,
                     title=title,
                     tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
                     x_axis_type=None, y_axis_type=None, min_border=1)

plot_lda.scatter(x=tsne_lda[:, 0], y=tsne_lda[:, 1],
                 color=colormap[_lda_keys][:num_example],
                 source=bp.ColumnDataSource({
                   "content": Issue_1[:num_example],
                   "topic_key": _lda_keys[:num_example]
                   }))

In [ ]:
save(plot_lda, 'tsne_lda_viz_{}_{}_{}_{}_{}.html'.format(num_topics, threshold, max_iteration, num_example, num_top_words))